In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from IPython.display import clear_output
import PIL as pl
import random
import pickle
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

In [2]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [3]:
# model = hub.KerasLayer("https://tfhub.dev/adityakane2001/regnety600mf_feature_extractor/1")

In [4]:
train_path = "./Train/"

In [5]:
types = np.load("dict.npy", allow_pickle = True).tolist()

In [6]:
data = pd.read_csv("train.csv")

In [7]:
data["tipo"] = data["Expected"].apply(lambda x : types[x.split(" ")[0]])

In [8]:
data["animal"] = data["Expected"].apply(lambda x : types[x.split(" ")[1]])

In [9]:
for file in os.listdir(train_path):
    if "jpeg" in file and "." not in file:
        new = file.replace("jpeg", ".jpeg")
        print(f"Changing : {file} to {new}")
        os.rename(train_path + file, train_path + file.replace("jpeg", ".jpeg"))

In [10]:
def change(x):
    if "jpeg" in x and "." not in x:
        return x.replace("jpeg", ".jpeg")
    return x

In [11]:
data["Id"] = data["Id"].apply(lambda x : change(x))

In [12]:
data = data.drop(2011)

In [13]:
data = data.reset_index().drop("index", axis = 1)

In [14]:
l = data.values.tolist()

In [15]:
random.shuffle(l)

In [16]:
data = pd.DataFrame(l, columns = ["Id", "Expected", "tipo", "animal"])

In [17]:
del l

In [19]:
shape_size = (224, 224)
size = len(data)

In [20]:
list_of_images = []
for i, file in enumerate(data["Id"]):
    
    print(f"Loading {i + 1} file of {size}")
    clear_output(wait = True)
    
    image = pl.Image.open(train_path + file).resize(shape_size)
    list_of_images.append(image)

Loading 3521 file of 3521


In [21]:
Images = np.zeros((len(data), shape_size[0], shape_size[1], 3), dtype = np.float32)

In [22]:
for i, image in enumerate(list_of_images):
    print(f"Loading {i + 1} file of {size}")
    clear_output(wait = True)
    
    try:
        array = np.asarray(image) / 255
        Images[i] = array
    except:
        array = (np.asarray(image).T[0 : 3] / 255).T
        Images[i] = array

Loading 3521 file of 3521


In [23]:
del list_of_images

In [27]:
data["animal"].unique()

array(['Buho', 'Mariposa', 'Caballo', 'Oso', 'Perro', 'Lobo', 'Cacique',
       'Ganso', 'Conejo', 'Paloma', 'Cormoran', 'Iguana', 'Cisne',
       'Lagarto', 'Leon', 'Pato', 'Elefante', 'Sapo', 'Lombriz',
       'Tarantula', 'Cocodrilo', 'Tortuga', 'Salamandra', 'Cabra', 'Gato',
       'Viuda negra', 'Tigre', 'Rana', 'Pavo', 'Camaleon', 'Serpiente',
       'Saltamontes'], dtype=object)

In [28]:
for col in data["animal"].unique():
    data[col] = data["animal"].apply(lambda x : int(x == col))

In [29]:
X_train, X_test, y_train, y_test = train_test_split(Images, data[data["animal"].unique()], stratify = data[data["animal"].unique()])

In [31]:
from keras_tuner import Hyperband

In [ ]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5")
])
m.build([None, 224, 224, 3])

In [45]:
def build_model(hp):
    model = tf.keras.Sequential()
  
    hp_units1 = hp.Int('units1', min_value = 64, max_value = 512, step = 4)
    hp_units2 = hp.Int('units2', min_value = 32, max_value = 512, step = 4)
    hp_units3 = hp.Int('units3', min_value = 6, max_value = 512, step = 4)
    
    model.add(tf.keras.layers.RandomRotation(0.2))
    model.add(tf.keras.layers.RandomFlip())
    model.add(tf.keras.layers.GaussianNoise(0.1))
    model.add(m)
    
    model.add(tf.keras.layers.Dense(units = hp_units1, activation = 'relu'))
    model.add(tf.keras.layers.Dense(units = hp_units2, activation = 'relu'))
    model.add(tf.keras.layers.Dense(units = hp_units3, activation = 'relu'))
    
    model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = tf.keras.metrics.BinaryAccuracy()
    )
    
    return model

In [46]:
import keras_tuner as kt

In [47]:
tuner = Hyperband(
    build_model,
    objective = "val_loss",
    max_epochs = 15,
    directory = 'tf_learning',
    project_name = 'transfer_learning',
    hyperband_iterations = 20
)

In [48]:
train_gen = DataGenerator(X_train, y_train, 32)
test_gen = DataGenerator(X_test, y_test, 32)

In [49]:
tuner.search(train_gen, epochs=10, validation_data = test_gen, callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)])

Trial 46 Complete [00h 00m 27s]
val_loss: 0.1586228907108307

Best val_loss So Far: 0.1404792219400406
Total elapsed time: 00h 29m 00s

Search: Running Trial #47

Hyperparameter    |Value             |Best Value So Far 
units1            |464               |496               
units2            |208               |292               
units3            |110               |106               
learning_rate     |0.001             |0.001             
tuner/epochs      |15                |15                
tuner/initial_e...|5                 |5                 
tuner/bracket     |2                 |2                 
tuner/round       |2                 |2                 
tuner/trial_id    |e56815c348fbc0a...|936c2e88e970026...

Epoch 6/15
83/83 [==============================] - 11s 102ms/step - loss: 0.1586 - binary_accuracy: 0.9627 - val_loss: 0.1432 - val_binary_accuracy: 0.9688
Epoch 7/15
83/83 [==============================] - 8s 94ms/step - loss: 0.1411 - binary_accuracy: 0.9688 - v


KeyboardInterrupt



In [50]:
b = tuner.get_best_hyperparameters()[0]

In [51]:
print(f'N1 : {b.get("units1")}\nN2 : {b.get("units2")}\nN3 : {b.get("units3")}\nLr : {b.get("learning_rate")}')

N1 : 496
N2 : 292
N3 : 106
Lr : 0.001


In [68]:
def make_model(hub_model):
    
    inputs = tf.keras.Input((None, None, 3))
    
    #Data Augmentation
    rot =tf.keras.layers.RandomRotation(0.2)(inputs)
    flip = tf.keras.layers.RandomFlip("horizontal")(rot)
    noise = tf.keras.layers.GaussianNoise(0.01)(flip)
    bright = tf.keras.layers.RandomBrightness((-0.5, 0.5), (0, 1))(noise)
    zoom = tf.keras.layers.RandomZoom((-0.5, 0.1), (-0.5, 0.1))(noise)
    
    x = hub_model(zoom)
    
    hidden_1 = tf.keras.layers.Dense(496, activation = "relu")(x)
    drop_1 = tf.keras.layers.Dropout(0.3)(hidden_1)
    hidden_2 = tf.keras.layers.Dense(292, activation = "relu")(drop_1)
    drop_2 = tf.keras.layers.Dropout(0.3)(hidden_2)
    hidden_3 = tf.keras.layers.Dense(106, activation = "relu")(drop_2)
    drop_3 = tf.keras.layers.Dropout(0.3)(hidden_3)
    
    outputs = tf.keras.layers.Dense(1, activation = "sigmoid")(drop_3)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = tf.keras.metrics.BinaryAccuracy()
    )
    
    return model

In [54]:
new_X_train = np.zeros((2640 * 2, 224, 224, 3))

In [55]:
for i in range(2640):
    new_X_train[i] = X_train[i]

In [56]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5")
])
m.build([None, 224, 224, 3])

In [61]:
m.trainable = False

In [62]:
tot = len(y_train.columns)

In [69]:
for cant, col in enumerate(y_train.columns):
    
    final_model = make_model(m)
    
    filter_ = y_train[col] == 1
    filtered = X_train[filter_]
    n = len(filtered)
    
    for i in range(2640, 2640 * 2):
        print(f"Training {col} {cant} of {tot}\nChanging {i + 1} of {2640 * 2}")
        clear_output(wait = True)
        aux = tf.keras.preprocessing.image.random_zoom(filtered[i % n], (0.8, 1.5), 0, 1, 2)
        aux = tf.keras.preprocessing.image.random_channel_shift(aux, 0.2, 2)
        aux = tf.keras.preprocessing.image.random_rotation(aux, 180, 0, 1, 2)
        new_X_train[i] = aux
    print(f"Training {col} {cant} of {tot}")
    new_values = y_train[col].values.tolist() + [1 for i in range(2640)]
    train_gen = DataGenerator(new_X_train, np.array([new_values]).T, 32)
    test_gen = DataGenerator(X_test, y_test[[col]], 32)
    
    final_model.fit(train_gen, validation_data = test_gen, epochs = 500, callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)])
    final_model.save(f"./Models/{col}.obj")

Training Lombriz 31 of 32
Epoch 1/500
165/165 [==============================] - 17s 77ms/step - loss: 0.2340 - binary_accuracy: 0.9402 - val_loss: 0.0089 - val_binary_accuracy: 0.9966
Epoch 2/500
165/165 [==============================] - 12s 71ms/step - loss: 0.0223 - binary_accuracy: 0.9934 - val_loss: 0.0104 - val_binary_accuracy: 0.9966
Epoch 3/500
165/165 [==============================] - 12s 70ms/step - loss: 0.0164 - binary_accuracy: 0.9955 - val_loss: 0.0103 - val_binary_accuracy: 0.9966
Epoch 4/500
165/165 [==============================] - 12s 70ms/step - loss: 0.0153 - binary_accuracy: 0.9955 - val_loss: 0.0034 - val_binary_accuracy: 0.9989
Epoch 5/500
165/165 [==============================] - 11s 69ms/step - loss: 0.0150 - binary_accuracy: 0.9953 - val_loss: 0.0069 - val_binary_accuracy: 0.9966
Epoch 6/500
165/165 [==============================] - 12s 70ms/step - loss: 0.0101 - binary_accuracy: 0.9977 - val_loss: 0.0104 - val_binary_accuracy: 0.9955
Epoch 7/500
165/165 

INFO:tensorflow:Assets written to: ./Models/Lombriz.obj\assets


In [32]:
l_models = os.listdir("./Animal models v2")

In [33]:
models = []
tot = len(l_models)
for cant, model in enumerate(l_models):
    print(f"Loading {model.replace('.obj', '')} {cant + 1} of {tot}")
    models.append(tf.keras.models.load_model(f"./Animal models v2/{model}"))
    clear_output(wait = True)

Loading Viuda negra 32 of 32


In [34]:
l_models = [x.replace('.obj', '') for x in l_models]

In [35]:
for model, name in zip(models, l_models):
    for layer in model.layers:
        layer._name = layer.name + name.replace(" ", "").lower()

In [41]:
for model in models:
    model.trainable = False

In [36]:
inverse_types = {}
for key in types:
    inverse_types[types[key]] = key

In [37]:
def get_prediction(prediction):
    return np.argmax(prediction)

In [62]:
inputs = tf.keras.Input((None, None, 3,))
resizing = tf.keras.layers.Resizing(224, 224)(inputs)
rescaling = tf.keras.layers.Rescaling(1/255)(resizing)
concat = tf.concat([m(rescaling, training = False) for m in models], 1)
# outputs = tf.keras.layers.Dense(32, activation = "softmax")(concat)

In [63]:
final_model = tf.keras.Model(inputs = inputs, outputs = concat)

In [64]:
final_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 resizing_2 (Resizing)          (None, 224, 224, 3)  0           ['input_5[0][0]']                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 224, 224, 3)  0           ['resizing_2[0][0]']             
                                                                                                  
 model_15 (Functional)          (None, 1)            6181994     ['rescaling_2[0][0]']      

In [44]:
test = pd.read_csv("test.csv")

In [45]:
get_type = {}
for ave in ["Buho", "Cacique", "Cisne", "Cormoran", "Ganso", "Paloma", "Pato", "Pavo"]:
    get_type[ave] = "Ave"
    
for anfibio in ["Rana", "Salamandra", "Sapo"]:
    get_type[anfibio] = "Anfibio"
    
for artropodo in ["Lombriz", "Mariposa", "Saltamontes", "Tarantula", "Viuda negra"]:
    get_type[artropodo] = "Artropodo"
    
for mamifero in ["Caballo", "Cabra", "Conejo", "Elefante", "Gato", "Leon", "Lobo", "Oso", "Perro", "Tigre"]:
    get_type[mamifero] = "Mamifero"
    
for reptil in ["Camaleon", "Cocodrilo", "Iguana", "Lagarto", "Serpiente", "Tortuga"]:
    get_type[reptil] = "Reptil"

In [38]:
predictions = []

In [46]:
for i, file in enumerate(test["Id"]):
    try:
        print(f"Openning {i} of {len(test)} - {file}")
        v = np.asarray(pl.Image.open("./Test/" + file))
        v = v.reshape((1, v.shape[0], v.shape[1], 3))
        p = final_model.predict(v)
        pred = l_models[get_prediction(p)]
        s = f"{inverse_types[get_type[pred]]} {inverse_types[pred]}"
        predictions.append([file, s])
        clear_output(wait  = True)
    except:
        print(f"Openning {i} of {len(test)} - {file}")
        v = np.asarray(pl.Image.open("./Test/" + file)).T[0 : 3].T
        v = v.reshape((1, v.shape[0], v.shape[1], 3))
        p = final_model.predict(v)
        pred = l_models[get_prediction(p)]
        s = f"{inverse_types[get_type[pred]]} {inverse_types[pred]}"
        predictions.append([file, s])
        clear_output(wait  = True)

Openning 1173 of 1174 - image_4229.jpg
1/1 [==============================] - 0s 293ms/step


In [47]:
pd.DataFrame(predictions, columns = ["Id", "Expected"]).to_csv("tercer_intento.csv", index = False)